# Example usage

Here's an example simulation, calculating income tax.

In [5]:
from policyengine_canada import Simulation

sim = Simulation(
    situation=dict(
        people=dict(
            person=dict(
                age=30,
                employment_income=20_000,
            )
        )
    )
)

sim.calculate("household_net_income")

array([18395.], dtype=float32)

Here's an example of using axes to calculate how variables relate to each other. Income tax is a progressive schedule (for an example), and people over 65 are exempt. The chart below plots income tax by income and age together.

In [6]:
from policyengine_canada import Microsimulation, Simulation
from policyengine_core.reforms import Reform
from policyengine_canada.model_api import *
import pandas as pd

YEAR = 2023


def make_hh(adults, children, child_age):
    people = dict(head=dict(age=30))
    members = ["head"]
    if adults == 2:
        people["spouse"] = dict(age=30)
        members += ["spouse"]
    for i in range(children):
        people[f"child{i}"] = dict(age=child_age)
        members += [f"child{i}"]
    return dict(
        people=people,
        households=dict(household=dict(members=members)),
        # Same impact for households with $250k+ income, so show up to $300k.
        # $1k increments.
        axes=[[dict(name="employment_income", count=301, min=0, max=300_000)]],
    )

l = []
for adults in [1]:
    for children in [0]:
        for child_age in [1]:
            hh = make_hh(adults, children, child_age)
            baseline_hh = Simulation(situation=hh)
            l.append(
                pd.DataFrame(
                    dict(
                        adults=adults,
                        children=children,
                        child_age=child_age,
                        # Reshape combined array to get head's varied earnings.
                        employment_income=baseline_hh.calculate(
                            "employment_income", YEAR
                        )[0 :: (adults + children)],
                        household_net_income=baseline_hh.calculate(
                            "household_net_income", YEAR
                        ),
                    )
                )
            )

df = pd.concat(l)
df




,adults,children,child_age,employment_income,household_net_income
0,1,0,1,0.0,0.000000
1,1,0,1,1000.0,850.000000
2,1,0,1,2000.0,1700.000000
3,1,0,1,3000.0,2550.000000
4,1,0,1,4000.0,3670.000000
...,...,...,...,...,...
296,1,0,1,296000.0,219628.078125
297,1,0,1,297000.0,220298.078125
298,1,0,1,298000.0,220968.078125
299,1,0,1,299000.0,221638.078125


In [7]:
import plotly.express as px

px.line(df, "employment_income", "household_net_income")